# Definition of Credit Risk and its importance in Risk Assessment

- What is Credit Risk and why is it important for Financial Institutions like banks, and fintech companies?

It is the probability of a client (company, institution, individual) to not be able to pay their installments and fall on credit. 

Reasons for that are (the list below is not exhaustive):

- Financial Problems and lack of liquidity
- Sales dropped last X months
- Recession and Covid

Finding that probability, companies can take measures to either reduce the credit limit, reject a request if necessary or chase merchants who are always late on their financial obligations. That way they can reduce the risk of losing cashflow. For any business to operate in a healthy manner, liquidity is the most important metric. 


Two statistical formulas for Credit Risk Analysis are:

1. Probability of Default (The likelihood of a borrower defaulting on a loan)
2. VaR (Value at Risk) in Credit Risk (A statistical measure used to assess the risk of loss in a financial portfolio (e.g. fintech's profit margin))


# Task 1

## 1.1 Load and merge dataframes from xlsx file with multiple Sheets

In [ ]:
import pandas as pd 
import numpy as np
from sklearn.preprocessing import LabelEncoder
import missingno as msno
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

applications = pd.read_excel('two_data.xlsx', sheet_name='applications')
company_data = pd.read_excel('two_data.xlsx', sheet_name='company_data')
payment_performance = pd.read_excel('two_data.xlsx', sheet_name='payment_performance')

applications

In [ ]:
company_data

In [ ]:
payment_performance

In [ ]:
print(applications.columns, company_data.columns, payment_performance.columns)

In [ ]:
# Joined at invoice level -- not company level to avoid duplications 
# Analysing risk per application

In [ ]:
app_to_companies = applications.merge(company_data, on='company_id', how='left')

payments_to_companies = payment_performance.merge(company_data, on='company_id', how='left')

joined_dfs = app_to_companies.merge(payment_performance, on=['company_id', 'invoice_id'], how='left')
joined_dfs

# Data Cleaning and Basic EDA

In [ ]:
# Understanding columns with nan values

In [ ]:
msno.matrix(joined_dfs)
plt.show()

In [ ]:
joined_dfs['application_date'] = pd.to_datetime(joined_dfs['application_date'], errors='coerce').dt.strftime('%Y-%m-%d')
joined_dfs['incorporation_date'] = pd.to_datetime(joined_dfs['incorporation_date'], errors='coerce').dt.strftime('%Y-%m-%d')

joined_dfs

In [ ]:
joined_dfs.isna().sum().sort_values(ascending=False)


In [ ]:
# fill categorical data with unknown since we cannot know the exact value missing

In [ ]:
joined_dfs['industry_segment'] = joined_dfs['industry_segment'].fillna('Unknown')
joined_dfs['postcode_area'] = joined_dfs['postcode_area'].fillna('Unknown')

joined_dfs['external_credit_rating_missing'] = joined_dfs['external_credit_rating'].isna().astype(int)
joined_dfs['external_credit_rating'] = joined_dfs['external_credit_rating'].fillna('Unknown')
joined_dfs

In [ ]:
# compute random date of the past for missing values.

In [ ]:
joined_dfs['incorporation_date'] = pd.to_datetime(joined_dfs['incorporation_date'], errors='coerce')\
    .fillna(pd.Timestamp('1970-01-01'))\
    .dt.strftime('%Y-%m-%d')

joined_dfs['payment_date'] = pd.to_datetime(joined_dfs['payment_date'], errors='coerce')\
    .fillna(pd.Timestamp('1970-01-01'))\
    .dt.strftime('%Y-%m-%d')
joined_dfs

In [ ]:
joined_dfs['incorporation_date_missing'] = (joined_dfs['incorporation_date'] == '1970-01-01').astype(int)
joined_dfs['payment_date_missing'] = (joined_dfs['payment_date'] == '1970-01-01').astype(int)
print(joined_dfs.columns)
joined_dfs

In [ ]:
# Check distributions of working capital and net worth before filling the nan values -- if outliers exist, median would help

In [ ]:

plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
sns.histplot(joined_dfs['working_capital'], kde=True, color="dodgerblue", bins=30, edgecolor="black")
plt.title('Distribution of Working Capital', fontsize=16)
plt.xlabel('Working Capital', fontsize=14)
plt.ylabel('Frequency', fontsize=14)


plt.show()

In [ ]:
plt.figure(figsize=(14, 7))

plt.subplot(1, 2, 1)
sns.histplot(joined_dfs['net_worth'], kde=True, color="mediumseagreen", bins=30, edgecolor="black")
plt.title('Distribution of Net Worth', fontsize=16)
plt.xlabel('Net Worth', fontsize=14)
plt.ylabel('Frequency', fontsize=14)


In [ ]:
# computing median of net worth and working capital since not all companies are having same sales 
# and net worth (industry sizes, locations, people's interests in purchases). 
# thus median is an appropriate metric in this case. 

In [ ]:

joined_dfs['payment_status'] = joined_dfs['payment_date'].isna().map({True: 'Unpaid', False: 'Paid'})
joined_dfs

In [ ]:
joined_dfs['working_capital'] = joined_dfs['working_capital'].fillna(joined_dfs['working_capital'].median())
joined_dfs['net_worth'] = joined_dfs['net_worth'].fillna(joined_dfs['net_worth'].median())

print(joined_dfs[['working_capital', 'net_worth']].isna().sum())

# Analyse Risk Score Column

In [ ]:
joined_dfs['risk_score'].isna().sum()

In [ ]:
summary_stats = joined_dfs['risk_score'].describe()
median_val = joined_dfs['risk_score'].median()

print(summary_stats)
print(median_val)

In [ ]:
# Outlier detection using IQR

Q1 = joined_dfs['risk_score'].quantile(0.25)
Q3 = joined_dfs['risk_score'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

outliers = joined_dfs[(joined_dfs['risk_score'] < lower_bound) | (joined_dfs['risk_score'] > upper_bound)]
print(len(outliers))

plt.figure(figsize=(10, 6))
sns.histplot(joined_dfs['risk_score'], bins=30, kde=True, color='dodgerblue', edgecolor='black')
plt.title("Distribution of Risk Score", fontsize=14)
plt.xlabel("Risk Score")
plt.ylabel("Frequency")
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Since there are outliers in the risk score, we can use median to impute the missing values

In [ ]:
joined_dfs['risk_score'] = joined_dfs['risk_score'].fillna(joined_dfs['risk_score'].median())
joined_dfs['risk_score'].isna().sum()

In [ ]:
default_rates = joined_dfs.groupby('industry_segment')['default_status'].sum().reset_index()

fig1 = px.bar(default_rates, x='industry_segment', y='default_status', 
              title='Default Rate Comparison by Industry Segment',
              labels={'default_status': 'Default Rate', 'industry_segment': 'Industry Segment'},
              color='industry_segment', color_continuous_scale='Viridis')

fig1.update_layout(xaxis_title='Industry Segment', yaxis_title='Default Rate')
fig1.show()

In [ ]:
default_rates = joined_dfs.groupby('industry_segment')['default_status'].mean().reset_index()

fig1 = px.bar(default_rates, x='industry_segment', y='default_status', 
              title='Default Rate Comparison by Industry Segment',
              labels={'default_status': 'Default Rate', 'industry_segment': 'Industry Segment'},
              color='industry_segment', color_continuous_scale='Viridis')

fig1.update_layout(xaxis_title='Industry Segment', yaxis_title='Default Rate')
fig1.show()

## More Visualisations for EDA & set to default those who paid their invoices after 90 days or never

In [ ]:
joined_dfs['due_date'] = pd.to_datetime(joined_dfs['due_date'])
joined_dfs['payment_date'] = pd.to_datetime(joined_dfs['payment_date'])

# Compute days late (for invoices that were paid)
joined_dfs['days_late'] = (joined_dfs['payment_date'] - joined_dfs['due_date']).dt.days

# Default status: True if unpaid or paid after 90+ days
joined_dfs['default_status'] = np.where(
    (joined_dfs['payment_date'].isna()) | (joined_dfs['days_late'] > 90),
    True,
    False
)

In [ ]:
default_rates = joined_dfs.groupby('industry_segment')['default_status'].mean().reset_index()

fig1 = px.bar(default_rates, x='industry_segment', y='default_status', 
              title='Default Rate Comparison by Industry Segment',
              labels={'default_status': 'Default Rate', 'industry_segment': 'Industry Segment'},
              color='industry_segment', color_continuous_scale='Viridis')

fig1.update_layout(xaxis_title='Industry Segment', yaxis_title='Default Rate')
fig1.show()

In [ ]:
fig2 = px.box(joined_dfs, x='default_status', y='risk_score', 
              title='Risk Score vs. Default Status',
              labels={'default_status': 'Default Status', 'risk_score': 'Risk Score'},
              color='default_status')

fig2.update_layout(xaxis_title='Default Status', yaxis_title='Risk Score')
fig2.show()

## Risk factor Interactions

In [ ]:
fig = px.scatter_3d(joined_dfs, x='requested_amount', y='working_capital', z='risk_score', color='default_status')
fig.show()

**Note**: High risk score in combination with low working capital despite the requested amount are indicators of default (credit risk)

## Some Time Series Analysis to Invoice Date and Default Status

In [ ]:
joined_dfs['invoice_date'] = pd.to_datetime(joined_dfs['invoice_date'])

df_true = joined_dfs[joined_dfs['default_status'] == 1]
df_false = joined_dfs[joined_dfs['default_status'] == 0]

df_true_grouped = df_true.groupby('invoice_date').size().reset_index(name='defaults_true')
df_false_grouped = df_false.groupby('invoice_date').size().reset_index(name='defaults_false')

df_merged = pd.merge(df_true_grouped, df_false_grouped, on='invoice_date', how='outer')

df_merged.fillna(0, inplace=True)

fig = px.line(df_merged, x='invoice_date', y=['defaults_true', 'defaults_false'],
              title="Default Rate Over Time (True vs False)",
              labels={'defaults_true': 'Defaults True', 'defaults_false': 'Defaults False'})

fig.update_traces(line=dict(color='red'), selector=dict(name='defaults_true'))
fig.update_traces(line=dict(color='green'), selector=dict(name='defaults_false'))

fig.show()


In [ ]:
joined_dfs['due_date'] = pd.to_datetime(joined_dfs['due_date'])

df_true = joined_dfs[joined_dfs['default_status'] == 1]
df_false = joined_dfs[joined_dfs['default_status'] == 0]

df_true_grouped = df_true.groupby('due_date').size().reset_index(name='defaults_true')
df_false_grouped = df_false.groupby('due_date').size().reset_index(name='defaults_false')

df_merged = pd.merge(df_true_grouped, df_false_grouped, on='due_date', how='outer')

df_merged.fillna(0, inplace=True)

fig = px.line(df_merged, x='due_date', y=['defaults_true', 'defaults_false'],
              title="Default Rate Over Due Time (True vs False)",
              labels={'defaults_true': 'Defaults True', 'defaults_false': 'Defaults False'})

fig.update_traces(line=dict(color='red'), selector=dict(name='defaults_true'))
fig.update_traces(line=dict(color='green'), selector=dict(name='defaults_false'))

fig.show()

**Note**: April to May 2022 we see some spikes in defaults suggesting that credit risk can be impacted by seasonality trends, 
or other macroeconomic risk factors such as recession risks, consumer confidence indecies, lack of sales to name a few

## Compute deltas of "days to pay" to see if this affects default status

In [ ]:
joined_dfs['payment_date'] = pd.to_datetime(joined_dfs['payment_date'])
joined_dfs['due_date'] = pd.to_datetime(joined_dfs['due_date'])

joined_dfs['delay_days'] = (joined_dfs['payment_date'] - joined_dfs['due_date']).dt.days

joined_dfs_filtered = joined_dfs.dropna(subset=['delay_days', 'default_status'])

fig_violin = px.violin(
    joined_dfs_filtered,
    y="delay_days",
    x="default_status",
    box=True,
    points="all",
    color="default_status",
    title="Payment Delay (Days) by Default Status",
    labels={"delay_days": "Days Late", "default_status": "Defaulted"},
)
fig_violin.show()

fig_box = px.box(
    joined_dfs_filtered,
    x="default_status",
    y="delay_days",
    color="default_status",
    title="Distribution of Payment Delays by Default Status",
    labels={"delay_days": "Days Late", "default_status": "Defaulted"},
)
fig_box.show()

summary = joined_dfs_filtered.groupby("default_status")["delay_days"].describe()
print("Summary statistics for payment delays by default status:")
print(summary)


Some companies which did not pay their amount surpassed the delayed days a lot (reaching up to a year almost)! -- Thus, we can conclude that specific delta could prompt us to expect default and so we should chase clients who take to long to respond to their financial obligations.

# 1.2 Compare construction segment to other industries with summary statistics

In [ ]:
joined_dfs.groupby('industry_segment')[['working_capital', 'net_worth']].describe()

In [ ]:
joined_dfs.groupby('industry_segment')['risk_score'].describe()

In [ ]:
joined_dfs['default_status'] = joined_dfs['default_status'].astype(int)

industry_default_rate = (
    joined_dfs.groupby('industry_segment')['default_status']
    .mean()
    .reset_index()
    .rename(columns={'default_status': 'default_rate'})
    .sort_values(by='default_rate', ascending=False)
)

fig = px.bar(
    industry_default_rate,
    x='industry_segment',
    y='default_rate',
    color='default_rate',
    color_continuous_scale='Reds',
    title='Default Rate by Industry Segment',
    labels={'default_rate': 'Default Rate', 'industry_segment': 'Industry Segment'}
)

fig.update_layout(xaxis_tickangle=-45)
fig.show()


# 1.3 Using payments data, calculate key risk metrics for the Construction vs. other segments.

In [ ]:
joined_dfs['invoice_date'] = pd.to_datetime(joined_dfs['invoice_date'])
joined_dfs['due_date'] = pd.to_datetime(joined_dfs['due_date'])
joined_dfs['payment_date'] = pd.to_datetime(joined_dfs['payment_date'])

joined_dfs['days_late'] = (joined_dfs['payment_date'] - joined_dfs['due_date']).dt.days

joined_dfs['days_outstanding'] = (joined_dfs['payment_date'] - joined_dfs['invoice_date']).dt.days

default_rate = joined_dfs.groupby('industry_segment')['default_status'].mean()

late_payment_rate = (joined_dfs[joined_dfs['days_late'] > 0].groupby('industry_segment').size() / joined_dfs.groupby('industry_segment').size())

avg_invoice_amount = joined_dfs.groupby('industry_segment')['invoice_amount'].mean()

avg_days_outstanding = joined_dfs.groupby('industry_segment')['days_outstanding'].mean()

risk_metrics = pd.DataFrame({
    'Default Rate': default_rate,
    'Late Payment Rate': late_payment_rate,
    'Avg Invoice Amount': avg_invoice_amount,
    'Avg Days Outstanding': avg_days_outstanding
})

construction_metrics = risk_metrics.loc[['Construct']]
other_metrics = risk_metrics.drop('Construct')

comparison_metrics = pd.concat([construction_metrics, other_metrics])

comparison_metrics


In [ ]:
sns.set(style="whitegrid")
plt.figure(figsize=(12, 6))

ordered_segments = (
    joined_dfs.groupby('industry_segment')['risk_score']
    .median()
    .sort_values()
    .index
)

sns.boxplot(x='industry_segment', y='risk_score', data=joined_dfs,
            order=ordered_segments, palette="Set3")

plt.axhline(y=5, color='red', linestyle='--', label='Default Threshold (risk_score = 5)')

plt.title("Risk Score Distribution by Industry Segment")
plt.xlabel("Industry Segment")
plt.ylabel("Risk Score")
plt.xticks(rotation=45)
plt.legend()

plt.tight_layout()
plt.show()


# 1.4 Identify any features from the data that appear correlated with default_status or
# suggest additional features to explore.

In [ ]:
categorical_features = ['industry_segment', 'legal_form', 'external_credit_rating', 'postcode_area', 'approved_amount',]

label_encoder = LabelEncoder()

for feature in categorical_features:
    joined_dfs[feature] = label_encoder.fit_transform(joined_dfs[feature].astype(str))

correlations_categorical = joined_dfs[categorical_features + ['default_status']].corr()
print(correlations_categorical)

sns.heatmap(correlations_categorical, annot=True, cmap='coolwarm', fmt='.2f')
plt.show()

In [ ]:
joined_dfs['liquidity_ratio'] = joined_dfs['working_capital'] / joined_dfs['net_worth']

joined_dfs['due_date'] = pd.to_datetime(joined_dfs['due_date'])
joined_dfs['payment_date'] = pd.to_datetime(joined_dfs['payment_date'])

joined_dfs['days_outstanding'] = (joined_dfs['payment_date'] - joined_dfs['due_date']).dt.days

In [ ]:
numerical_features = ['requested_amount', 'risk_score', 'invoice_amount', 'external_credit_rating', 'liquidity_ratio', 'days_outstanding']

correlations = joined_dfs[numerical_features + ['default_status']].corr()
print(correlations)

sns.heatmap(correlations, annot=True, cmap='coolwarm', fmt='.2f')
plt.show()

In [ ]:
joined_dfs.columns

## More Visualisations

In [ ]:
fig3 = px.scatter(joined_dfs, x='invoice_amount', y='days_outstanding', 
                  color='default_status', 
                  title='Invoice Amount vs. Days Outstanding',
                  labels={'invoice_amount': 'Invoice Amount', 'days_outstanding': 'Days Outstanding'})

fig3.update_layout(xaxis_title='Invoice Amount', yaxis_title='Days Outstanding')
fig3.show()

In [ ]:
fig4 = px.box(joined_dfs, x='default_status', y='liquidity_ratio', 
              title='Liquidity Ratio vs. Default Status',
              labels={'default_status': 'Default Status', 'liquidity_ratio': 'Liquidity Ratio'},
              color='default_status')

fig4.update_layout(xaxis_title='Default Status', yaxis_title='Liquidity Ratio')
fig4.show()

In [ ]:
fig5 = px.box(joined_dfs, x='default_status', y='working_capital', 
              title='Working Capital vs. Default Status',
              labels={'default_status': 'Default Status', 'working_capital': 'Working Capital'},
              color='default_status')

fig5.update_layout(xaxis_title='Default Status', yaxis_title='Working Capital')
fig5.show()


**Note**: higher working capital and network, thus liquidity ratio, less likely to default

In [ ]:
fig9 = px.box(joined_dfs, x='default_status', y='net_worth', 
              title='Net Worth vs. Default Status',
              labels={'default_status': 'Default Status', 'net_worth': 'Net Worth'},
              color='default_status')

fig9.update_layout(xaxis_title='Default Status', yaxis_title='Net Worth')
fig9.show()


In [ ]:
fig8 = px.box(joined_dfs, x='default_status', y='external_credit_rating', 
              title='External Credit Rating vs. Default Status',
              labels={'default_status': 'Default Status', 'external_credit_rating': 'External Credit Rating'},
              color='default_status')

fig8.update_layout(xaxis_title='Default Status', yaxis_title='External Credit Rating')
fig8.show()


In [ ]:
fig6 = px.histogram(joined_dfs, x='days_late', color='default_status',
                    title='Days Late Distribution by Default Status',
                    labels={'days_late': 'Days Late'})

fig6.update_layout(xaxis_title='Days Late', yaxis_title='Count', barmode='overlay')
fig6.show()

# Check for outliers in other columns and determine the impact

In [ ]:
numerical_columns = ['requested_amount', 'approved_amount','invoice_amount', 
                     'days_outstanding', 'days_late', 'liquidity_ratio', 'working_capital', 'net_worth']

def detect_outliers_iqr(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
    return outliers

outliers_dict = {}
for column in numerical_columns:
    outliers_dict[column] = detect_outliers_iqr(joined_dfs, column)

for column, outliers in outliers_dict.items():
    print(f"Outliers for {column}:")
    print(outliers[[column]])
    print("\n")

In [ ]:
plt.figure(figsize=(15, 10))
for i, column in enumerate(numerical_columns, 1):
    plt.subplot(3, 3, i)
    sns.boxplot(data=joined_dfs, x=column)
    plt.title(f'Box plot for {column}')
plt.tight_layout()
plt.show()

# Class Imbalance Analysis for Default Status Binary Target Variable

In [ ]:
fig = px.pie(joined_dfs, names='default_status', title="Class Distribution of Default Status")
fig.show()

#### **Note**: the default status data is highly imbalanced suggesting for a SMOTE technique or upsampling of true values for a better ML prediction 

Relationships between the default status and the rest of features:

- No strong linear correlations with default_status — suggests the relationships are likely non-linear or interacting.

- industry_segment has the highest correlation, hinting that sector-based segmentation may be useful.

- external_credit_rating and risk_score are only weakly predictive on their own. 

- Features like invoice_amount, liquidity_ratio, and requested_amount alone aren't helpful, but they may be more useful when combined as ratios. 


To improve prediction and gain a deeper understanding, incorporating financial health indicators, payment behavior data, market trends, and specific segment-level data could help in refining the model for default prediction.

- Segment-Specific Interactions:

industry_segment * risk_score (are certain industries riskier?)

- Create Ratios:

invoice_amount / liquidity_ratio



- Financial Health Metrics:

Debt-to-Equity Ratio: This ratio could help understand the financial leverage of a company. Companies with higher debt may be more prone to default.

Profitability Metrics: Profit margins or Return on Assets (ROA) could provide insight into how well a company can sustain itself and avoid defaults.


- Payment Behavior:

Number of Late Payments: Tracking late payments over time or categorizing them into "frequent late payers" could help predict defaults more accurately.

Invoice Payment Frequency: Companies that consistently delay or pay invoices in installments might show higher default risk.

- Time Factors:

Duration of Company Existence: A company that has been around for a longer period might be less likely to default. This could be captured by calculating the number of years since incorporation.


- External Market Data:

Economic Indicators: Factors such as GDP growth, inflation, or sector-specific economic data could provide a broader context to help predict defaults.

Credit Ratings of Key Customers/Partners: If available, analyzing the creditworthiness of the company's main business partners might provide additional predictive power.

- Legal Form and Ownership Structure:

Ownership Type (e.g., family-owned vs. publicly listed): The ownership structure could affect the default likelihood due to different levels of oversight, financial stability, and risk tolerance.

Legal Form: Corporations, partnerships, and sole proprietorships might have different risk profiles. Exploring this might reveal insights into default likelihood.

- Market Trends:

Economic Cycles: For example, during recessions, companies may experience higher default rates. Incorporating such features could provide a more accurate prediction model.

# TASK 2

**2.1**

Evaluate if the current risk assessment (as indicated by risk_score and decision) appears adequate for the Construction segment.
   
Default Risk (Adequacy of Risk Score):

Default rate is high enough (0.79%), in comparison to other industries suggesting that companies in Construction default.

All, the late payment rate is high (55.6%), meaning that while they pay, they often pay late.


Decision Process:

Risk scoring should distinguish between late payers and defaulters. Construction may just need longer payment cycles, not stricter credit limits.

Given these observations, the current risk_score and decision might not be fully adequate for assessing the risk in the Construction sector. 


A possible approach would be to use a segmentation-based rule for adjusting credit limits based on specific metrics derived from the data. Here's a proposed approach:

Segment by Default Risk: Divide the Construction segment into high-risk and low-risk sub-segments based on their historical default rate and late payment behavior.

- High-Risk Sub-Segment: Companies with a default rate above certain threshold and/or a late payment rate above 50%.

- Low-Risk Sub-Segment: Companies with a default rate below certain threshold and a late payment rate below 50%.

Adjust Credit Limits: Based on this segmentation:

- For High-Risk Companies: Lower their credit limits or set stricter terms (e.g., shorter payment terms, increased collateral).

- For Low-Risk Companies: Maintain or slightly increase their credit limits, as their historical performance suggests they are more likely to meet their obligations.


How to Evaluate This Rule in Production:

A/B Testing:

- Compare default and repayment patterns (test group) vs. control group.

Track Metrics:

- Default rate

- Late payment rate

- Recovery rate

- Credit utilization




**2.2**

Proposed Data-Driven Rule for the Construction Segment

Given that the Construction segment shows a default rate of (0.79%) and a high late payment rate (55.6%), the current risk assessment may be overly conservative. To better align credit decisions with actual risk, we propose the following data-driven segmentation rule:

For applicants in the Construction industry segment, we can apply an adjusted credit limit if the following conditions are met:

1. Late payments are consistently within a tolerable range (e.g., < 60 days overdue),

2. No recorded defaults in the past 12 months,

3. Liquidity ratio is above a threshold (e.g., > 1.2), and

4. External credit rating is not worse than a predefined floor (e.g., equivalent of B or above).

This rule introduces flexibility for segments with late but ultimately reliable payers. 

### An SQL example in our GCP bigquery database could look like below:

```sql
SELECT 
    customer_id,
    industry_segment,
    risk_score,
    liquidity_ratio,
    external_credit_rating,
    default_status,
    AVG(days_outstanding) AS avg_days_outstanding,
    COUNT(CASE WHEN days_outstanding > 60 THEN 1 END) AS late_payments_over_60_days
FROM 
    credit_data
WHERE 
    industry_segment = 'Construct'
GROUP BY 
    customer_id, industry_segment, risk_score, liquidity_ratio, external_credit_rating, default_status
HAVING 
    default_status = False AND
    liquidity_ratio > 1.2 AND
    external_credit_rating IN ('AAA', 'AA', 'A', 'BBB', 'BB', 'B') AND
    late_payments_over_60_days = 0;



**2.3**

The proposed rule aims to refine credit limit assessments for Construction segment clients by filtering for strong liquidity, good credit ratings, and consistent payment behavior. Here's why this makes sense:

1. Higher Late Payment Rate in Construction
Construction has a late payment rate of ~55.6%, which is among the highest across all sectors.

This indicates that the default is high (0.79%) and delays in payments are systemically common in this segment.

Therefore, isolating clients with no significant delays (no payments >60 days late) is a meaningful way to filter for operational discipline.

2. Distinction Between Defaulters and Non-Defaulters
The average days_outstanding for non-defaulters is -1894 days, showing early payments (possibly due to credit notes or accounting quirks).

For defaulters, the average delay is +176 days, with even the 25th percentile at 116 days late — far worse than the general population.

This validates using long delays as an early warning signal.

3. Liquidity Ratio as a Key Differentiator
While we don’t have it broken out by default status here, applying a threshold > 1.2 helps identify customers with healthier short-term solvency.

This supports the idea that liquidity adds a buffer against late or missed payments.

4. Risk Score Median Suggests Conservative Baseline
The median risk score is 3.06 — using this as a reference, our query can focus on Construction clients whose behavior does not match typical high-risk profiles.

Even though risk_score alone doesn’t capture default directly, it's useful for ranking within a filtered subset.

5. External Credit Rating Adds Assurance
By including only firms rated BBB or above, the rule emphasizes external validation of creditworthiness.

This helps mitigate Construction’s operational risks by balancing internal and external data sources.

**2.4**

Summary of Evaluation and Deployment Steps for Rule-Based System (SQL-based):

a. Pilot test using historical data and A/B testing to evaluate its initial effectiveness.

b. Evaluate metrics like default rate, credit limits, and customer behavior. (Default Rate: After applying the rule, check if the default rate in the Construction segment is decreasing.)

c. Test edge cases to ensure the rule works under various conditions. (are there cases where a customer might not fit the rule but still poses a risk?)

d. Monitor performance after launch, and set up automated alerts for anomalies. (To monitor metrics like default rate, credit limit adjustments, and payment behaviors. Any significant shifts in these metrics should trigger alerts.)

e. Collect feedback to refine the rule, and ensure it's achieving the desired outcomes. (If the business is seeing unexpected outcomes (e.g., defaults are not reducing), reconsider the thresholds or logic used in the rule.)

f. Regular performance audits to ensure the rule stays relevant over time. (If customer behavior or risk factors change over time, you may need to adjust the rule's parameters (e.g., change the risk_score threshold or modify the conditions))

g. Document and version control to track rule changes and make updates easily. 